In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from utils.data_preprocess import *
from models import *

In [2]:
data = pd.read_csv('./data/ETHBTC-5m-data.csv')
data = data_split(data, 3)[0]

In [3]:
WINDOW_SIZE = 24

In [4]:
raw_data = DataLabeling(data, WINDOW_SIZE)
train_df, val_df, test_df = train_val_test_split(raw_data.labelled_data, train_size=0.6, val_size=0.2, test_size=0.2)
Data = DataPreprocess(train_df, val_df, test_df, window_size=WINDOW_SIZE, label_size=1, label_columns=['Label'], shift=0, batch_size=8)

In [5]:
train_data = Data.train.prefetch(tf.data.AUTOTUNE)
val_data = Data.val.prefetch(tf.data.AUTOTUNE)

In [6]:
strong_model = CDT_1D_model(WINDOW_SIZE, 5).model

In [7]:
strong_model.fit(train_data,
                 epochs=1000,
                 validation_data=val_data,
                 steps_per_epoch=len(train_data),
                 validation_steps=int(0.15 * len(val_data)),
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2)])

Epoch 1/1000
11337/11337 [==============================] - 106s 9ms/step - loss: 0.4398 - accuracy: 0.8863 - val_loss: 0.3998 - val_accuracy: 0.8982 - lr: 0.0010
Epoch 2/1000
11337/11337 [==============================] - 98s 9ms/step - loss: 0.4336 - accuracy: 0.8863 - val_loss: 0.3997 - val_accuracy: 0.8982 - lr: 0.0010
Epoch 3/1000
 2913/11337 [======>.......................] - ETA: 1:14 - loss: 0.5114 - accuracy: 0.8551

In [16]:
strong_model.evaluate(Data.test)

473/473 [==============================] - 3s 6ms/step - loss: 0.4326 - accuracy: 0.8865


[0.43261733651161194, 0.8864508867263794]

In [18]:
weak_model = MLP_model(WINDOW_SIZE, 5).model
weak_model.fit(train_data,
                 epochs=1000,
                 validation_data=val_data,
                 steps_per_epoch=len(train_data),
                 validation_steps=int(0.15 * len(val_data)),
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2),
                            create_model_checkpoint(model_name=weak_model.name, save_path='data_without_TIs')])

Epoch 1/1000
1418/1418 [==============================] - 16s 10ms/step - loss: 0.5291 - accuracy: 0.8858 - val_loss: 0.4028 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 2/1000
1418/1418 [==============================] - 16s 11ms/step - loss: 0.4456 - accuracy: 0.8863 - val_loss: 0.4019 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 3/1000
1418/1418 [==============================] - 15s 10ms/step - loss: 0.4359 - accuracy: 0.8863 - val_loss: 0.4014 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 4/1000
1418/1418 [==============================] - 15s 11ms/step - loss: 0.4331 - accuracy: 0.8863 - val_loss: 0.4009 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 5/1000
1418/1418 [==============================] - 14s 10ms/step - loss: 0.4321 - accuracy: 0.8863 - val_loss: 0.4006 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 6/1000
1418/1418 [==============================] - 16s 11ms/step - loss: 0.4319 - accuracy: 0.8863 - val_loss: 0.4004 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 7/1000
1418/1418 [==========

In [19]:
weak_model.evaluate(Data.test)

473/473 [==============================] - 3s 7ms/step - loss: 0.4326 - accuracy: 0.8865


[0.432639479637146, 0.8864508867263794]

In [1]:
from tensorflow.keras import Sequential, layers, Model, Input
def model():
    input = Input(shape=[24, 5])
    x = layers.LSTM(300, return_sequences=True)(input)
    x = layers.LSTM(200, return_sequences=True)(x)
    x = layers.LSTM(100, return_sequences=True)(x)
    x = layers.LSTM(50, return_sequences=True)(x)
    x = layers.LSTM(20)(x)
    x = layers.Dense(500, activation='relu')(x)
    x = layers.Dropout(0.7)(x)
    output = layers.Dense(3, activation='softmax')(x)
    model = Model(input, output)
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
test = model()
test.fit(train_data, epochs=5)

Epoch 1/5
1418/1418 [==============================] - 48s 21ms/step - loss: 0.4419 - accuracy: 0.8861
Epoch 2/5
1418/1418 [==============================] - 32s 23ms/step - loss: 0.4377 - accuracy: 0.8863
Epoch 3/5
1418/1418 [==============================] - 30s 21ms/step - loss: 0.4360 - accuracy: 0.8863
Epoch 4/5
1418/1418 [==============================] - 31s 22ms/step - loss: 0.4353 - accuracy: 0.8863
Epoch 5/5
1418/1418 [==============================] - 34s 24ms/step - loss: 0.4347 - accuracy: 0.8863
